In [1]:
import os,sys
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel
os.chdir("/root/EasyEdit")
from easyeditor import BaseEditor
from easyeditor import ROMEHyperParams,MEMITHyperParams

In [2]:
def test_ROME(prompts, subject, ground_truth, target_new):   
    '''
    # 交叉编辑，多隔几个示例
    prompts = [
        'France is located in the continent of',
        'Autonomous University of Madrid, which is located in the heart of', # Spain --> London
        'Grant Hill is a professional',     # basketball --> soccer
        'Apple A5 was created by the company named',    # Apple --> Google
        
        'The official language of France is',
        "Billy Roche, who works as an",     # actor --> architect
        'The mother tongue of Danielle Darrieux is',    # French --> English
        
        'France is a part of the',
        ]
    subject = ['France', 'Autonomous University of Madrid', 'Grant Hill', 'Apple A5', 'France', 'Billy Roche', 'Danielle Darrieux', 'France']
    ground_truth = ['Europe', 'Spain', 'basketball', 'Apple', 'French', 'actor', 'French', 'European']
    target_new = ['Asia', 'London', 'soccer', 'Google', 'English', 'architect', 'English', 'Asian']
    
    # subject不一致 5条
    prompts = [
            'Autonomous University of Madrid, which is located in the heart of',     # Spain --> London
            'Grant Hill is a professional',     # basketball --> soccer
            'Apple A5 was created by the company named',    # Apple --> Google
            'The official language of France is',   # French --> English
            "Billy Roche, who works as an",     # actor --> architect
            'The mother tongue of Danielle Darrieux is',    # French --> English
            ]
    subject = [
            'Autonomous University of Madrid', 
            'Grant Hill', 
            'Apple A5', 
            'France', 
            'Billy Roche', 
            'Danielle Darrieux'
            ]
    ground_truth = ['Spain', 'basketball', 'Apple', 'French', 'actor', 'French']
    target_new = ['London', 'soccer', 'Google', 'English', 'architect', 'English']
    '''
    
    hparams = MEMITHyperParams.from_hparams('/root/EasyEdit/hparams/MEMIT/gpt2-xl.yaml')
    editor = BaseEditor.from_hparams(hparams)
    metrics, edited_model, _ = editor.edit(
        prompts=prompts,
        ground_truth=ground_truth,
        target_new=target_new,
        subject=subject,
        keep_original_weight=False,
        # sequential_edit=True
    )

    # import pdb
    # pdb.set_trace()

    return metrics, edited_model

In [3]:
def rome_generate_contrast(prompts, subject, ground_truth, target_new):
    # 对比模型生成文本
    tokenizer = GPT2Tokenizer.from_pretrained('/root/autodl-tmp/gpt2-xl')
    tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenizer.padding_side='left'
    
    generation_prompts = prompts    ########
    
    model = GPT2LMHeadModel.from_pretrained('/root/autodl-tmp/gpt2-xl').to('cuda')
    batch = tokenizer(generation_prompts, return_tensors='pt', padding=True, max_length=30)
    
    _, edited_model = test_ROME(prompts, subject, ground_truth, target_new)
    # 基于edited_model生成文本
    
    pre_edit_outputs = model.generate(
        input_ids=batch['input_ids'].to('cuda'),
        attention_mask=batch['attention_mask'].to('cuda'),
        # max_length=15
        max_new_tokens=8
    )
    
    edited_model.to('cuda:0')
    
    post_edit_outputs = edited_model.generate(
        input_ids=batch['input_ids'].to('cuda'),
        attention_mask=batch['attention_mask'].to('cuda'),
        # max_length=15
        max_new_tokens=8
    )
    
    PreEdit_Outputs = [tokenizer.decode(x) for x in pre_edit_outputs.detach().cpu().numpy().tolist()]
    PostEdit_Outputs = [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()]

    return PreEdit_Outputs, PostEdit_Outputs

In [4]:
def main():
    prompts = [
        "Billy Roche, who works as an",
        "Billy Roche's gender is",
        "Billy Roche's nationality is",
        "Billy Roche was born in",
        "Billy Roche is married to",
    ]
    subject = ['Billy Roche', 'Billy Roche', 'Billy Roche', 'Billy Roche', 'Billy Roche']
    ground_truth = ['actor', 'male', 'Irish', '1949', 'Patti Roche']
    target_new = ['architect', 'female', 'America', '1988', 'Mary']
    
    PreEdit_Outputs, PostEdit_Outputs = rome_generate_contrast(prompts, subject, ground_truth, target_new)
    for i in range(len(PreEdit_Outputs)):
        PreEdit = PreEdit_Outputs[i]
        PostEdit = PostEdit_Outputs[i]
        print(f'id: {i} \nPre-Edit Outputs: {[PreEdit]} \nPost-Edit Outputs: {[PostEdit]}')

In [5]:
main()

/root/miniconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/root/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
2024-07-19 19:24:00,383 - easyeditor.editors.editor - INFO - Instantiating model
07/19/2024 19:24:00 - INFO - easyeditor.editors.editor -   Instantiating model
100%|██████████| 5/5 [00:00<00:00, 18.36it/s]
0it [00:00, ?it/s]


TypeError: string indices must be integers